In [1]:
%matplotlib inline

import json

from datetime import datetime
from math import ceil

from keras.applications.xception import Xception, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
DATA_PATH = 'data/'
MODEL_PATH = 'models/'

BATCH_SIZE = 128

## Load data

The data has been organized into train and validation foldes with each car make in its separate subfolder.

The training data has been augmented beforehand with 90 degree rotations.

In [3]:
train_generator = image.ImageDataGenerator(
    rotation_range=5, 
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    rescale = 1/255.,
    preprocessing_function=preprocess_input,
)

In [4]:
train_batches = train_generator.flow_from_directory(
    DATA_PATH + 'train/',
    target_size=(299, 299),
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

Found 15655 images belonging to 10 classes.


In [5]:
valid_generator = image.ImageDataGenerator(
    rescale = 1/255.,
    preprocessing_function=preprocess_input,
)

In [6]:
valid_batches = valid_generator.flow_from_directory(
    DATA_PATH + 'valid/',
    target_size=(299, 299),
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

Found 1967 images belonging to 10 classes.


In [7]:
num_classes = train_batches.num_classes

 ## Build model
 
 Using Xception V1 model as a base.

In [8]:
def save_model(model, train_batches):
    now = datetime.now()
    
    model_name = MODEL_PATH + f'carnet-{now.date().isoformat()}_{now.hour*60 + now.minute}'

    model.save_weights(model_name + '-weights.h5')
    with open(model_name + '-model.json', 'w') as f:
        f.write(model.to_json())
    
    with open(model_name + '-classes.json', 'w') as f:
        json.dump(train_batches.class_indices, f)

In [9]:
def load_model(model_name):
    with open(MODEL_PATH + model_name + '-model.json') as f:
        model = model_from_json(f.read())
        
    model.load_weights(MODEL_PATH + model_name + '-weights.h5')
    
    return model

In [10]:
base_model = Xception(weights='imagenet', include_top=False)

In [11]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [12]:
model = Model(inputs=base_model.input, outputs=predictions)

Leave only the last two added layers trainable.

In [13]:
for layer in base_model.layers:
    layer.trainable = False

In [14]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

## Training

In [16]:
model.fit_generator(
    train_batches,
    steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
    validation_data=valid_batches,
    validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
    epochs=1,
)

Epoch 1/1
123/123 [==============================] - 584s 5s/step - loss: 2.0357 - acc: 0.2865 - val_loss: 2.2427 - val_acc: 0.1871


In [17]:
save_model(model, train_batches)

In [18]:
model.optimizer.lr = 0.1

In [19]:
model.fit_generator(
    train_batches,
    steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
    validation_data=valid_batches,
    validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
    epochs=10,
)

Epoch 1/10
123/123 [==============================] - 583s 5s/step - loss: 1.8579 - acc: 0.3676 - val_loss: 1.9906 - val_acc: 0.2933
Epoch 2/10
123/123 [==============================] - 534s 4s/step - loss: 1.7753 - acc: 0.3965 - val_loss: 1.8570 - val_acc: 0.3371
Epoch 3/10
123/123 [==============================] - 533s 4s/step - loss: 1.7297 - acc: 0.4082 - val_loss: 1.7408 - val_acc: 0.4128
Epoch 4/10
123/123 [==============================] - 539s 4s/step - loss: 1.6909 - acc: 0.4224 - val_loss: 1.6883 - val_acc: 0.4199
Epoch 5/10
123/123 [==============================] - 538s 4s/step - loss: 1.6715 - acc: 0.4322 - val_loss: 1.6954 - val_acc: 0.4052
Epoch 6/10
123/123 [==============================] - 543s 4s/step - loss: 1.6455 - acc: 0.4420 - val_loss: 1.6534 - val_acc: 0.4342
Epoch 7/10
123/123 [==============================] - 538s 4s/step - loss: 1.6156 - acc: 0.4518 - val_loss: 1.6428 - val_acc: 0.4296
Epoch 8/10
123/123 [==============================] - 536s 4s/step - 

In [20]:
save_model(model, train_batches)

In [21]:
model.optimizer.lr = 0.01

In [22]:
model.fit_generator(
    train_batches,
    steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
    validation_data=valid_batches,
    validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
    epochs=10,
)

Epoch 1/10
123/123 [==============================] - 591s 5s/step - loss: 1.5804 - acc: 0.4623 - val_loss: 1.6378 - val_acc: 0.4260
Epoch 2/10
123/123 [==============================] - 535s 4s/step - loss: 1.5645 - acc: 0.4690 - val_loss: 1.6242 - val_acc: 0.4408
Epoch 3/10
123/123 [==============================] - 542s 4s/step - loss: 1.5536 - acc: 0.4717 - val_loss: 1.5970 - val_acc: 0.4545
Epoch 4/10
123/123 [==============================] - 538s 4s/step - loss: 1.5396 - acc: 0.4781 - val_loss: 1.6113 - val_acc: 0.4423
Epoch 5/10
123/123 [==============================] - 536s 4s/step - loss: 1.5279 - acc: 0.4823 - val_loss: 1.5943 - val_acc: 0.4479
Epoch 6/10
123/123 [==============================] - 541s 4s/step - loss: 1.5321 - acc: 0.4814 - val_loss: 1.5846 - val_acc: 0.4606
Epoch 7/10
123/123 [==============================] - 531s 4s/step - loss: 1.5124 - acc: 0.4884 - val_loss: 1.5852 - val_acc: 0.4637
Epoch 8/10
123/123 [==============================] - 536s 4s/step - 

In [23]:
save_model(model, train_batches)

In [24]:
model.optimizer.lr = 0.001

In [ ]:
for i in range(20):
    print(f'{i+1}:')

    model.fit_generator(
        train_batches,
        steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
        validation_data=valid_batches,
        validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
        epochs=10,
    )
    
    save_model(model, train_batches)

1:
Epoch 1/10
123/123 [==============================] - 581s 5s/step - loss: 1.4823 - acc: 0.5039 - val_loss: 1.5781 - val_acc: 0.4514
Epoch 2/10
123/123 [==============================] - 537s 4s/step - loss: 1.4861 - acc: 0.4978 - val_loss: 1.5419 - val_acc: 0.4814
Epoch 3/10
123/123 [==============================] - 533s 4s/step - loss: 1.4855 - acc: 0.4976 - val_loss: 1.5750 - val_acc: 0.4540
Epoch 4/10
123/123 [==============================] - 543s 4s/step - loss: 1.4789 - acc: 0.5006 - val_loss: 1.5669 - val_acc: 0.4677
Epoch 5/10
123/123 [==============================] - 533s 4s/step - loss: 1.4663 - acc: 0.5043 - val_loss: 1.5279 - val_acc: 0.4748
Epoch 6/10
123/123 [==============================] - 537s 4s/step - loss: 1.4591 - acc: 0.5075 - val_loss: 1.5897 - val_acc: 0.4514
Epoch 7/10
123/123 [==============================] - 534s 4s/step - loss: 1.4574 - acc: 0.5087 - val_loss: 1.5900 - val_acc: 0.4565
Epoch 8/10
123/123 [==============================] - 538s 4s/step

123/123 [==============================] - 540s 4s/step - loss: 1.3402 - acc: 0.5475 - val_loss: 1.4850 - val_acc: 0.4875
Epoch 3/10
123/123 [==============================] - 537s 4s/step - loss: 1.3404 - acc: 0.5468 - val_loss: 1.4979 - val_acc: 0.4881
Epoch 4/10
123/123 [==============================] - 544s 4s/step - loss: 1.3385 - acc: 0.5497 - val_loss: 1.4878 - val_acc: 0.4921
Epoch 5/10
123/123 [==============================] - 538s 4s/step - loss: 1.3434 - acc: 0.5435 - val_loss: 1.4884 - val_acc: 0.4855
Epoch 6/10
123/123 [==============================] - 543s 4s/step - loss: 1.3349 - acc: 0.5488 - val_loss: 1.4881 - val_acc: 0.4931
Epoch 7/10
123/123 [==============================] - 534s 4s/step - loss: 1.3334 - acc: 0.5482 - val_loss: 1.4895 - val_acc: 0.4972
Epoch 8/10
123/123 [==============================] - 538s 4s/step - loss: 1.3315 - acc: 0.5499 - val_loss: 1.4719 - val_acc: 0.4982
Epoch 9/10
123/123 [==============================] - 538s 4s/step - loss: 1.327